In [1]:
import pandas as pd
books = pd.read_csv('books_cleaned.csv')

In [3]:
books['categories'].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Conspiracies,1
475,Brothers and sisters,1
476,Rock musicians,1
477,Community life,1


In [4]:
books['categories'].value_counts().reset_index().query('count > 50')

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Philosophy,117
6,Religion,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [6]:
category_mapping = {
    'Fiction' : 'Fiction',
    'Juvenile Fiction' : "Children's Fiction",
    'Biography & Autobiography' : 'Nonfiction',
    'History' : 'Nonfiction',
    'Literary Criticism' : 'Nonfiction',
    'Philosophy' : 'Nonfiction',
    'Religion' : 'Nonfiction',
    'Science' : 'Nonfiction',
    'Comics & Graphic Novels' : 'Fiction',
    'Juvenile Nonfiction' : "Children's Nonfiction",
    'Poetry' : 'Fiction',
    'Drama' : 'Fiction'
}

books['categories'] = books['categories'].map(category_mapping)

In [7]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age_of_book,words_count,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,21.0,199,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web : A Novel,Charles Osborne;Agatha Christie,NaN,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,25.0,205,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,32.0,57,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,NaN,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,23.0,45,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,NaN,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,23.0,75,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [9]:
from transformers import pipeline

fiction_categories = ['Fiction', 'Nonfiction']

pipe = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

Device set to use mps:0


In [11]:
sequence = books.loc[books['categories'] == 'Fiction', 'description'].reset_index(drop=True)[0]

In [12]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [15]:
import numpy as np

def predict_category(seq, category):
    preds = pipe(seq, category)
    max_index = np.argmax(preds['scores'])
    max_label = preds['labels'][max_index]
    return max_label

In [25]:
from tqdm import tqdm

known_categories = []
predicted_categories = []

for i in tqdm(range(0, 300)):
    seq = books.loc[books['categories'] == 'Fiction', 'description'].reset_index(drop=True)[i]
    predicted_categories += [predict_category(seq, fiction_categories)]
    known_categories += ['Fiction']

100%|██████████| 300/300 [08:06<00:00,  1.62s/it]


In [26]:
for i in tqdm(range(0, 300)):
    seq = books.loc[books['categories'] == 'Nonfiction', 'description'].reset_index(drop=True)[i]
    predicted_categories += [predict_category(seq, fiction_categories)]
    known_categories += ['Nonfiction']

100%|██████████| 300/300 [07:27<00:00,  1.49s/it]


In [27]:
predictions_df = pd.DataFrame({'actual': known_categories, 'predicted': predicted_categories})
predictions_df

,actual,predicted
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction
...,...,...
595,Nonfiction,Nonfiction
596,Nonfiction,Fiction
597,Nonfiction,Nonfiction
598,Nonfiction,Nonfiction


In [28]:
predictions_df['correct_predictions'] = (np.where(predictions_df['actual'] == predictions_df['predicted'], 1, 0))

In [29]:
predictions_df['correct_predictions'].sum() / len(predictions_df)

np.float64(0.7783333333333333)

In [30]:
# Predicting missing categories
isbns = []
predicted_categories = []

missing_categories = books.loc[books['categories'].isna(), ['isbn13', 'description']].reset_index(drop=True)

In [32]:
for i in tqdm(range(0, len(missing_categories))):
    seq = missing_categories['description'][i]
    predicted_categories += [predict_category(seq, fiction_categories)]
    isbns.append(missing_categories['isbn13'][i])

100%|██████████| 1454/1454 [48:23<00:00,  2.00s/it] 


In [34]:
missing_predictions_df = pd.DataFrame({'isbn13': isbns, 'predicted_categories': predicted_categories})
missing_predictions_df

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1449,9788125026600,Nonfiction
1450,9788171565641,Fiction
1451,9788172235222,Fiction
1452,9788173031014,Nonfiction


In [37]:
books = pd.merge(books, missing_predictions_df, on='isbn13', how='left')
books['categories'] = np.where(books['categories'].isna(), books['predicted_categories'], books['categories'])
books = books.drop(columns='predicted_categories')

In [38]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age_of_book,words_count,tagged_description,predicted_categories_x,predicted_categories_y
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,21.0,199,9780002005883 A NOVEL THAT READERS and critics...,NaN,NaN
1,9780002261982,0002261987,Spider's Web : A Novel,Charles Osborne;Agatha Christie,Fiction,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,25.0,205,9780002261982 A new 'Christie for Christmas' -...,Fiction,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,32.0,57,"9780006178736 A memorable, mesmerizing heroine...",NaN,NaN
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Nonfiction,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,23.0,45,9780006280897 Lewis' work on the nature of lov...,Nonfiction,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Nonfiction,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,23.0,75,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Fiction,http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,22.0,288,9788172235222 On A Train Journey Home To North...,Fiction,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Nonfiction,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,23.0,63,9788173031014 This book tells the tale of a ma...,Nonfiction,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Fiction,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,22.0,117,9788179921623 Wisdom to Create a Life of Passi...,Fiction,Fiction
5195,9788185300535,8185300534,I Am that : Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Nonfiction,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,26.0,174,9788185300535 This collection of the timeless ...,NaN,NaN


In [39]:
books.to_csv('categorised_books.csv', index=False)